In [1]:
import os
import sys
# you probably don't need this, but I had trouble importing packages because I used a virtual env
sys.path.append(os.path.join(os.getcwd(), 'venv\Lib\site-packages'))

from dotenv import load_dotenv
from knowledge.elastic_client import ElasticConfig
import networkx as nx
import matplotlib.pyplot as plt

from utils import bfs_subgraph, dfs_paths, graph_from_paths

In [2]:
class GraphComposer:
    def __init__(self, graph_dict, linking_instructions, join_graphs, lenses):
        self.graph_dict = graph_dict
        self.linking_instructions = linking_instructions
        self.composed_graph = nx.DiGraph()
        self.join_graphs = join_graphs
        self.lenses = lenses

    def compose_graphs(self):
        for linking_instruction in self.linking_instructions:
            self._link_graphs(linking_instruction)

        for source_name, f_graph in self.join_graphs['sources']:
            self._one_to_many_joins(source_name, f_graph)


    def apply_lenses(self, lense_names, graph):
        # create a sub graph based on the lense
        G: nx.DiGraph = graph or self.composed_graph
        nodes = G.nodes(data=True)
        new_nodes = []
        for lense_name in lense_names:
            lense = self.lenses[lense_name]
            for n in nodes:
                if lense['source'](n):
                    if lense['condition'](n):
                        new_nodes.append(n)              
                else:
                    new_nodes.append(n)
        
        T = nx.DiGraph()
        for n in new_nodes:
            T.add_node(n[0], **n[1])
            
        filtered_edges = [(u, v) for (u, v) in G.edges() if u in T.nodes() and v in T.nodes()]
        T.add_edges_from(filtered_edges)
        return T


    def get_composed_graph(self):
        return self.composed_graph


    def _one_to_many_joins(self, source_name, f_graph):
        for n in f_graph.nodes(data=True):
            if not n[1].get('joins'):
                continue

            joins = n[1]['joins']
            for join in joins:
                self._join_graph(source_name, n, join)


    def _join_graph(self, source_name, n, join):
        index, filter_fn, sub_joins, graph = self._get_join_info(join)
        if not graph:
            return

        filtered_nodes = [node for node in graph.nodes(data=True) if filter_fn(node[1]['props'])]
        for nt in filtered_nodes:
            new_source = source_name + ":" + n[0]
            new_target = index + ":" + nt[0]
            self.composed_graph.add_edge(new_source, new_target)
        
        if sub_joins:
            new_index, filter_fn, _, graph = self._get_join_info(sub_joins)
            new_filtered_nodes = [node for node in graph.nodes(data=True) if filter_fn(node[1]['props'])]
            for nt in filtered_nodes:
                for nft in new_filtered_nodes:
                        new_source = index + ":" + nt[0]
                        new_target = new_index + ":" + nft[0]
                        self.composed_graph.add_edge(new_source, new_target)

    
    def _get_join_info(self, join):
        index = join['index']
        filter_fn = join['filter']
        sub_joins = join.get('join')
        graph = self.join_graphs['on'].get(index)
        return index,filter_fn,sub_joins,graph


    def _link_graphs(self, linking_instruction):
        target = linking_instruction['target']
        source = linking_instruction['source']
        link = linking_instruction['link']
        target_graph = self.graph_dict[target]
        source_graph = self.graph_dict[source]
        
        for source_node in source_graph.nodes(data=True):
            target_nodes = filter(lambda x: x if link(source_node[1], x[1]) else None, target_graph.nodes(data=True))
            for target_node in target_nodes:
                new_source = source + ":" + source_node[0]
                new_target = target + ":" + target_node[0]
                source_data = { **source_node[1], 'source': source }
                target_data = { **target_node[1], 'source': target }
                self.composed_graph.add_node(new_source, **source_data)
                self.composed_graph.add_node(new_target, **target_data)
                
                self.composed_graph.add_edge(new_source, new_target)

In [3]:
current_env = 'remote' if os.path.exists('.env') else 'local'

if current_env == 'local':
    dotenv_path = '.env.local'
else:
    dotenv_path = '.env'

load_dotenv(dotenv_path)

esc = ElasticConfig(
    https=current_env != 'local',
    username=os.getenv('ELASTIC_USERNAME'),
    password=os.getenv('ELASTIC_PASSWORD'),
    url=os.getenv('ELASTIC_URL'),
    port=os.getenv('ELASTIC_PORT'))

In [4]:
actions_graph = nx.Graph()
# todo action -> item (pickaxe) -> block
mine_filters = [
    {'index': 'items', 'filter': lambda x: 'pickaxe' in x.get('name'), 
     'join': {'index': 'blocks', 'filter': lambda x: x.get('material') == 'mineable/pickaxe' } }]
actions_graph.add_node('mine', props = { 'name': 'mine' }, joins=mine_filters)
actions_graph.add_node('collect', props = { 'name': 'collect' }, joins=[{ 'index': 'items', 'filter': lambda x: x } ])
actions_graph.add_node('fight', props = { 'name':'fight' }, joins=[{'index': 'entities', 'filter': lambda x: x['type'] == 'Hostile mobs'} ])
actions_graph.add_node('hunt', props = { 'name':'hunt' }, joins=[{ 'index': 'entities', 'filter': lambda x: x['type'] == 'Passive mobs' }])
actions_graph.add_node('eat', props = { 'name':'eat' }, joins=[{ 'index': 'foods', 'filter': lambda x: x['food_points'] > 0 }])
actions_graph.add_node('craft', props = { 'name':'craft' }, joins=[{'index': 'recipe', 'filter': lambda x: x }])
actions_graph.add_node('trade', props={ 'name': 'trade' }, joins=[{ 'index': 'trade', 'filter': lambda x: 'bid' in x['name'] or 'ask' in x['name'] }])

agent_graph = nx.Graph()
agent_graph.add_node('bill', props={ 'actions': ['mine', 'collect', 'fight', 'hunt', 'eat', 'craft', 'smelt', 'trade'] })

inventory_graph = nx.Graph()
inventory_graph.add_node('wooden_pickaxe', props = { 'name': 'wooden_pickaxe', 'quantity': 1 })

goals_graph = nx.Graph()
goals_graph.add_node('make_money', props={'name': 'make_money', 'objective': ['money'] })

trade_graph = nx.Graph()
trade_graph.add_node('bid', props={ 'name': 'bid' }, joins=[{'index': 'trade', 'filter': lambda x: 'debit' == x['name'] }]) # money -> debit -> all items
trade_graph.add_node('ask', props={ 'name': 'ask' }, joins=[{'index': 'trade', 'filter': lambda x: 'credit' == x['name']  }]) # inventory items -> credit -> money
trade_graph.add_node('debit', props={'name': 'debit' }, joins=[{ 'index': 'items', 'filter': lambda x: x } ]) 
trade_graph.add_node('credit', props={'name': 'credit' }, joins=[{ 'index': 'inventory', 'filter': lambda x: x }]) # could be , { 'index': 'trade', 'filter': lambda x: 'money' in x['name'] }
trade_graph.add_node('money', props={'name': 'money' }, joins=[{ 'index': 'trade', 'filter': lambda x: 'debit' == x['name'] or 'credit' == x['name'] } ])

In [12]:
from knowledge.graph.load import get_graph_dict

def ins(x):
    print(x)
    return x

linkings = [
    {
        'source': 'blocks',
        'target': 'items',
        'link': lambda b, i: i['props']['name'] in [x['name'] for x in b['props']['drops']]
    },
    {
        'source': 'blocks',
        'target': 'items',
        'link': lambda b, i: i['props']['name'] in [x['name'] for x in b['props']['requires']]
    },
    {
        'source': 'items',
        'target': 'foods',
        'link': lambda s, t: s['props']['name'] == t['props']['name'],
    },
    {
        'source': 'entities',
        'target': 'entity_loot',
        'link': lambda e, l: e['props']['name'] == l['props']['entity']
    },
    {
        'source': 'entity_loot',
        'target': 'items',
        'link': lambda el, i: i['props']['name'] in [x['item'] for x in el['props']['drops']]
    },
    {
        'source': 'items',
        'target': 'smelting',
        'link': lambda i, s: i['props']['name'] == s['props']['result']
    },
    # this one is quite complicated
    # {
    #     'source': 'smelting',
    #     'target': 'items',
    #     'link': lambda s, i: i['props']['name'] in ins(s['props']['ingredient']['item'])
    # },
    {
        'source': 'items',
        'target': 'recipes',
        'link': lambda i, r: i['props']['name'] in [x['provides']['name'] for x in r['props']['items']]
    },
    {
        'source': 'recipes',
        'target': 'items',
        'link': lambda r, i: i['props']['name'] in [z['name'] for x in r['props']['items'] for z in x['needs']]
    },
    {
        'source': 'agent',
        'target': 'goals',
        'link': lambda s, t: True # all goals are linked to all actions
    },
    {
        'source': 'goals',
        'target': 'actions',
        'link': lambda s, t: True # all goals are linked to all actions
        # todo get agent actions t['props']['name'] in s['props']['actions']
    },
]

graphs = get_graph_dict(esc)
graphs['actions'] = actions_graph
graphs['agent'] = agent_graph
graphs['inventory'] = inventory_graph
graphs['goals'] = goals_graph
graphs['trade'] = trade_graph

one_to_many_join_graphs = { 
    'sources': [('actions', actions_graph), ('trade', trade_graph)], 
    'on': graphs 
}

def lens(item):
    if not item[1].get('source'):
        return False
    return 'items' in item[1]['source']

lenses = {
    'only_inventory_mining_items': { 
        #'source': lambda x: 'items' in x[1]['source'], 
        'source': lambda x: lens(x),
        'condition': lambda x: 'pickaxe' not in x[1]['props']['name'] or x[1]['props']['name'] in [n[1]['props']['name'] for n in inventory_graph.nodes(data=True)] 
    }
}

composer = GraphComposer(graphs, linkings, one_to_many_join_graphs, lenses)
composer.compose_graphs()

composed_graph = composer.get_composed_graph()

In [6]:
def visualize_graph(graph):
    #initialze Figure
    plt.figure(num=None, figsize=(80, 80), dpi=300)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx(graph, pos, with_labels=True)
    plt.draw()
    plt.show()

In [ ]:
visualize_graph(composed_graph)

In [ ]:
nx.shortest_path(composed_graph, "blocks:granite", "recipes:2")
#min([composed_graph.degree(x) for x in composed_graph.nodes()])

['blocks:granite', 'items:granite', 'recipes:2']

In [ ]:
tree = bfs_subgraph(composed_graph, source='actions:mine')
visualize_graph(tree)

In [ ]:
tree = bfs_subgraph(composed_graph, source='actions:mine')
tree = composer.apply_lenses(['only_inventory_mining_items'], tree)
visualize_graph(tree)

In [10]:
# I wasn't able to run this...
# it maxed out at about 23GB of RAM 
# and crashed after 30minutes.

paths = dfs_paths(composed_graph, "goals:make_money")
paths = [path for path in paths if "trade:credit" in path]
G = graph_from_paths(paths)
visualize_graph(G)

MemoryError: 